In [1]:
# Read the candidate csv file into a dataframe

import pandas as pd

CSV_PATH = "../results/NUMT_candidates.csv"

df = pd.read_csv(CSV_PATH)

In [4]:
# Cluster reads that fall into the same bin

BIN_SIZE = 500

df["bin_start"] = (df["ref_start"] // BIN_SIZE) * BIN_SIZE

grouped = df.groupby(["ref_name", "bin_start", "is_reverse"])

clustered = grouped.agg(
    num_reads=("query_name", "count"),
    avg_ref_start=("ref_start", "mean"),
    avg_mate_start=("mate_start", "mean")
).reset_index()

print(clustered)

     ref_name  bin_start  is_reverse  num_reads  avg_ref_start  avg_mate_start
0           1    1300000       False          1      1300043.0         16075.0
1           1    1332000        True          2      1332031.0         11714.0
2           1    1356000       False          1      1356407.0         15220.0
3           1    1562500        True          1      1562969.0          3920.0
4           1    2504000       False          1      2504078.0         12920.0
...       ...        ...         ...        ...            ...             ...
1711        X  151904500        True          1    151904803.0         11676.0
1712        X  154465500        True          1    154465570.0          7393.0
1713        Y   13704500        True          1     13704513.0          9212.0
1714        Y   13718500        True          1     13718730.0         13277.0
1715        Y   20727000        True          1     20727335.0          2944.0

[1716 rows x 6 columns]
